# Dash Financiero

### Configuración del entorno

1. Se creó un ambiente de python en una carpeta llamada dash_task2. Para este caso, usaremos conda con los siguientes códigos:

```PowerShell
conda create --name taskdash_venv python=3.9
```

```PowerShell
conda activate taskdash_venv

cd "C:\Users\varga\OneDrive\Escritorio\dash_task2"
```

2. Instale los siguientes requeremientos con los codigos a continuación en la consolda de conda:
```PowerShell
pip install dash
pip install jupyter-dash
pip install pandas
```

3. Crea un archivo .py en visual code dashclass.py

4. Luego, ejecutas el siguiente código y copias lo que te da en el navegador

```Powershell
python.exe .\dashclass.py
```

## Construcción de la aplicación

Se realizaron cambios al código inical. A continuación de expone detallado del código y, finalmente, se expresan los cambios realizados

### Instalación de requerimientos y cargue de datos

Se importan todas las librerias necesarias para el desarollo del dash

```Python
from dash import Dash, dcc, html, Input, Output
import colorlover as cl
import pandas as pd
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
```

```Python
df = pd.read_csv('https://raw.githubusercontent.com/lihkir/Uninorte/main/AppliedStatisticMS/DataVisualizationRPython/Lectures/Python/PythonDataSets/dash-stock-ticker-demo.csv')
```

Creación de la aplicación Dash y configuración del tema y las paletas de colores

```Python
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY]) 
```

Creación de una paleta de colores personalizada
```Python
custom_colorscale = cl.scales['9']['qual']['Set1']
```

### Diseño y estructura de la aplicación Dash
El código registrado a continuación corresponde al Front-end de una aplicación Dash para explorar datos financieros. A continuación, describiré y explicaré los cambios realizados en el código editado en comparación con el código original:

* Cambios en el estilo del encabezado:

En el código original, el encabezado "Finance Explorer" tenía un color de texto "rgba(117, 117, 117, 0.95)". En el código editado, el color del texto se cambió a blanco mediante el atributo 'color': "#FFFFFF". Esto hace que el texto del encabezado sea blanco en lugar de gris.

Se añadió un nuevo atributo 'background-color': "#333" en el código editado para cambiar el color de fondo del encabezado a un tono gris oscuro (#333). En el código original, no se especificaba un color de fondo, por lo que tenía un fondo transparente o predeterminado.

* Agregar gráficos:

En el código editado, se agregaron dos componentes dcc.Graph con los identificadores 'candlestick-graph' y 'bollinger-graph' en el div con el id 'graphs'. Estos gráficos, que no estaban presentes en el código original, se utilizarán para mostrar un gráfico de velas y un gráfico de bandas de Bollinger respectivamente.



```Python
app.layout = html.Div([
    html.Div([
        html.H2('Finance Explorer',
                style={'display': 'inline',
                       'font-size': '2.65em',
                       'font-weight': 'bolder',
                       'font-family': 'Product Sans',
                       'color': "#FFFFFF",  # Cambio del color del texto a blanco
                       'margin-top': '20px',
                       'margin-bottom': '0',
                       'background-color': "#333"  # Cambio del color de fondo
                       }),
        html.Img(src="https://s3-us-west-1.amazonaws.com/plotly-tutorials/logo/new-branding/dash-logo-by-plotly-stripe.png",
                style={
                    'height': '100px',
                    'float': 'right'
                },
        ),
    ]),
    dcc.Dropdown(
        id='stock-ticker-input',
        options=[{'label': s[0], 'value': str(s[1])}
                 for s in zip(df.Stock.unique(), df.Stock.unique())],
        value=['YHOO', 'GOOGL'],
        multi=True
    ),
    html.Div(id='graphs', children=[
        dcc.Graph(id='candlestick-graph'),  # Gráfico de velas
        dcc.Graph(id='bollinger-graph')  # Gráfico de bandas de Bollinger
    ])
], className="container")

```

### Definición de una función para calcular las bandas de Bollinger

La función `bbands` calcula las bandas de Bollinger para un conjunto de datos de precios financieros. Para ello, toma tres parámetros: la serie de precios (`price`), el tamaño de la ventana para calcular la media móvil (`window_size`), y el número de desviaciones estándar a considerar (`num_of_std`). Primero, se calcula la media móvil y la desviación estándar de los precios utilizando la ventana especificada. Luego, se utilizan estos valores para determinar la banda superior y la banda inferior de Bollinger. La banda superior se obtiene sumando `num_of_std` veces la desviación estándar a la media móvil, mientras que la banda inferior se calcula restando `num_of_std` veces la desviación estándar a la media móvil. La función devuelve la media móvil, la banda superior y la banda inferior, que son fundamentales en el análisis técnico de los mercados financieros para evaluar la volatilidad y las posibles señales de compra o venta.

```Python
def bbands(price, window_size=10, num_of_std=2):
    rolling_mean = price.rolling(window=window_size).mean()
    rolling_std  = price.rolling(window=window_size).std()
    upper_band = rolling_mean + (rolling_std * num_of_std)
    lower_band = rolling_mean - (rolling_std * num_of_std)
    return rolling_mean, upper_band, lower_band

```

### Actualización de tablas y gráficos

En el código editado, la función `update_graph` tiene dos salidas (Outputs): 'candlestick-graph' y 'bollinger-graph', que corresponden a dos gráficos diferentes: el gráfico de velas (candlestick) y el gráfico de bandas de Bollinger. Estos dos gráficos se actualizarán por separado.
En lugar de devolver una lista de gráficos, la función crea diccionarios para cada uno de los gráficos (candlestick_graph y bollinger_graph). Cada diccionario contiene un campo 'data' y un campo 'layout', que definen la información y la configuración del gráfico.
Se utiliza la biblioteca Plotly (mediante go.Candlestick y go.Scatter) para configurar y crear los gráficos, lo que proporciona un mayor control sobre su apariencia y comportamiento

El código proporcionado configura dos tipos de gráficos, uno de velas (candlestick) y otro de bandas de Bollinger, que se utilizarán en una aplicación financiera. Cada gráfico se configura para mostrar datos específicos de acciones seleccionadas por el usuario en un menú desplegable. El gráfico de velas representa los precios de apertura, máximos, mínimos y cierre en un período de tiempo, y se personaliza con colores diferenciados para velas alcistas y bajistas. El gráfico de bandas de Bollinger representa una banda calculada en función de los precios de cierre y muestra cómo los precios se desvían de una media móvil.

Estos gráficos se configuran dinámicamente para cada acción seleccionada por el usuario, lo que permite una exploración detallada de los datos financieros. El código se asegura de que las trazas de cada acción se agrupen en la leyenda y se gestionen las leyendas en el gráfico de bandas de Bollinger. 

```Python
@app.callback([
    Output('candlestick-graph', 'figure'),
    Output('bollinger-graph', 'figure')
], [
    Input('stock-ticker-input', 'value')
])
def update_graph(tickers):
    if not tickers:
        return {}, {}

    candlestick_graph = {'data': [], 'layout': {}}
    bollinger_graph = {'data': [], 'layout': {}}

    for i, ticker in enumerate(tickers):
        dff = df[df['Stock'] == ticker]

        # Configuración del gráfico de velas (candlestick)
        candlestick_trace = go.Candlestick(
            x=dff['Date'],
            open=dff['Open'],
            high=dff['High'],
            low=dff['Low'],
            close=dff['Close'],
            name=ticker,
            legendgroup=ticker,
            increasing_line_color=custom_colorscale[0],
            decreasing_line_color=custom_colorscale[1]
        )

# Configuración del gráfico para las bandas de Bollinger
        bollinger_traces = go.Scatter(
            x=dff['Date'],
            y=bbands(dff['Close'])[0],
            mode='lines',
            line=dict(width=1, color=custom_colorscale[(i * 2) % len(custom_colorscale)]),
            hoverinfo='x+y',  # Muestra el valor en el eje x (Fecha) y el valor en el eje y (Valor de la banda)
            legendgroup=ticker,
            showlegend=True if i == 0 else False,
            name='{} - bollinger bands'.format(ticker)
        )

        candlestick_graph['data'].append(candlestick_trace)
        bollinger_graph['data'].append(bollinger_traces)

    candlestick_graph['layout'] = {
        'margin': {'b': 0, 'r': 10, 'l': 60, 't': 0},
        'legend': {'x': 0}
    }

    return candlestick_graph, bollinger_graph
```

### Código completo

````Python
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
import colorlover as cl


app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

custom_colorscale = cl.scales['9']['qual']['Set1']

df = pd.read_csv('https://raw.githubusercontent.com/lihkir/Uninorte/main/AppliedStatisticMS/DataVisualizationRPython/Lectures/Python/PythonDataSets/dash-stock-ticker-demo.csv')

app.layout = html.Div([
    html.Div([
        html.H2('Finance Explorer',
                style={'display': 'inline',
                       'font-size': '2.65em',
                       'font-weight': 'bolder',
                       'font-family': 'Product Sans',
                       'color': "#FFFFFF",
                       'margin-top': '20px',
                       'margin-bottom': '0',
                       'background-color': "#333"
                       }),
        html.Img(src="https://s3-us-west-1.amazonaws.com/plotly-tutorials/logo/new-branding/dash-logo-by-plotly-stripe.png",
                style={
                    'height': '100px',
                    'float': 'right'
                },
        ),
    ]),
    dcc.Dropdown(
        id='stock-ticker-input',
        options=[{'label': s[0], 'value': str(s[1])}
                 for s in zip(df.Stock.unique(), df.Stock.unique())],
        value=['YHOO', 'GOOGL'],
        multi=True
    ),
    html.Div(id='graphs', children=[
        dcc.Graph(id='candlestick-graph'),
        dcc.Graph(id='bollinger-graph')
    ])
], className="container")

def bbands(price, window_size=10, num_of_std=2):
    rolling_mean = price.rolling(window=window_size).mean()
    rolling_std  = price.rolling(window=window_size).std()
    upper_band = rolling_mean + (rolling_std * num_of_std)
    lower_band = rolling_mean - (rolling_std * num_of_std)
    return rolling_mean, upper_band, lower_band

@app.callback([
    Output('candlestick-graph', 'figure'),
    Output('bollinger-graph', 'figure')
], [
    Input('stock-ticker-input', 'value')
])
def update_graph(tickers):
    if not tickers:
        return {}, {}

    candlestick_graph = {'data': [], 'layout': {}}
    bollinger_graph = {'data': [], 'layout': {}}

    for i, ticker in enumerate(tickers):
        dff = df[df['Stock'] == ticker]

        # Configuración del gráfico de velas (candlestick)
        candlestick_trace = go.Candlestick(
            x=dff['Date'],
            open=dff['Open'],
            high=dff['High'],
            low=dff['Low'],
            close=dff['Close'],
            name=ticker,
            legendgroup=ticker,
            increasing_line_color=custom_colorscale[0],
            decreasing_line_color=custom_colorscale[1]
        )

        # Configuración del gráfico para las bandas de Bollinger
        bollinger_traces = go.Scatter(
            x=dff['Date'],
            y=bbands(dff['Close'])[0],
            mode='lines',
            line=dict(width=1, color=custom_colorscale[(i * 2) % len(custom_colorscale)]),
            hoverinfo='x+y',  # Muestra el valor en el eje x (Fecha) y el valor en el eje y (Valor de la banda)
            legendgroup=ticker,
            showlegend=True if i == 0 else False,
            name='{} - bollinger bands'.format(ticker)
        )

        candlestick_graph['data'].append(candlestick_trace)
        bollinger_graph['data'].append(bollinger_traces)

    candlestick_graph['layout'] = {
        'margin': {'b': 0, 'r': 10, 'l': 60, 't': 0},
        'legend': {'x': 0}
    }

    return candlestick_graph, bollinger_graph

if __name__ == '__main__':
    app.run_server(debug=True, host='0.0.0.0', port=9000)



```

## Test y Dokerización

Se emplearon los mismos procedimientos que en la sección anterior asociada a la app para mapas